<a href="https://colab.research.google.com/gist/agalitsyna/9b0b172ce83261e94ff40062f2652d22/hichew_api_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hi! This is a **notebook** for DNA-DNA interactions analysis 


Notabook is a piece of programming code with annotations and a diary of your actions at the same time. In this notebook we will go into details of time series analysis for DNA-DNA interactions.

It was designed so that you can understand of basics in multiple fields simultaneously:

- Colab Google service for programming code development

- Jupyter notebooks for programming demonstration

- Python programming

- Theory of DNA-DNA interactions capture in biological samples (Hi-C)

- Hi-C data analysis and manipulation

As you can see, these are multiple problems that we tackle here, and it's okay to **question how it works** and **pause and look into steps until you got them**.

Let's start from simple things to get acquainted with Colab and notebooks.


In [ ]:
### This is a piece of Python code that you can run by clicking the "play" button or pressing Shift+Enter
print("Hello, SMTBologist!")

In [ ]:
# Ah, if you see some text starting with "#", it is a comment and not a code itself

In [ ]:
# If you want to learn more about function or variable in Python, you can always ask for help. Just type:
print?

### Set up the coding environment

In [ ]:
# Download and install code
! git clone https://github.com/encent/hichew && cd hichew && pip install -e .
! git clone https://github.com/nvictus/lavaburst.git && cd lavaburst && make build -f Makefile && make install -f Makefile

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import important packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

### What are DNA-DNA interactions and how do we measure them?

Hi-C is an experimental technique that outputs DNA_DNA interactions in the nuclei of the tested cells:



In [ ]:
%%html
<iframe src="https://slides.com/agalicina/time-series-dna-graph/embed" width="576" height="420" scrolling="no" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe>

Let's analyse Hi-C data!

In [ ]:
import cooler # Load the package to work with Hi-C
import networkx as nx # Visualize the graph

In [ ]:
!ls /content/drive/MyDrive/SMTB_data/

In [ ]:
cool_file = cooler.Cooler('/content/drive/MyDrive/SMTB_data/3-4h_repl_merged_5kb.mcool::/resolutions/20000')

In [ ]:
cool_file.info

In [ ]:
cool_file.chromnames

In [ ]:
interactions = cool_file.matrix(as_pixels=True).fetch('chr2L')

In [ ]:
interactions.head()

In [ ]:
matrix = cool_file.matrix(as_pixels=False).fetch('chr2L')

In [ ]:
plt.figure(figsize=[5,5])
plt.imshow(matrix[0:30, 0:30], cmap='Reds', vmax=0.2)

Let's visualize it as a graph:

In [ ]:
G = nx.convert_matrix.from_numpy_matrix(matrix[0:30, 0:30])

In [ ]:
plt.figure(figsize=[14,7])
subax1 = plt.subplot(121)
nx.draw(G, with_labels=True, font_weight='bold')
subax2 = plt.subplot(122)
nx.draw_shell(G, with_labels=True, font_weight='bold')

In [ ]:
plt.figure(figsize=[14,14])

elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 0.1]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.1]

pos = nx.spring_layout(G, seed=7)  # positions for all nodes - seed for reproducibility

# nodes
nx.draw_networkx_nodes(G, pos, node_size=700)

# edges
nx.draw_networkx_edges(G, pos, edgelist=elarge, width=2)
nx.draw_networkx_edges(G, pos, edgelist=esmall, width=1, alpha=0.5, edge_color="b")

# labels
nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")

print('')

### Analysis of a set of experiments

In [ ]:
import hichew

In [ ]:
from hichew.calling import boundaries, domains, clusters
from hichew.compute import normalize, d_scores, insulation_scores, silhouette

In [ ]:
from hichew.loader import cool_files

In [ ]:
from hichew.plot import clusters_dynamics, viz_opt_curves, viz_tads, _pca, _tsne

In [ ]:
%matplotlib inline

### Load cool files:

In [ ]:
matrices, coolers = cool_files('/content/drive/MyDrive/SMTB_data/', resolution=5000, chromnames=['chrX'])

In [ ]:
matrices.keys()

# TAD boundaries

### Call boundaries:

In [ ]:
BOUNDARIES_df, BOUNDARIES_df_opt, \
BOUNDARIES_stats, BOUNDARIES_opt_windows = boundaries(matrices, coolers, label='3-4h_repl_merged_5kb', 
                                                      expected_tad_size=60000, chromnames=['chrX'], 
                                                      filtration='custom', bs_thresholds={'3-4h_repl_merged_5kb': 0.35})

In [ ]:
BOUNDARIES_df_opt.shape

### Visualize opt curves:

In [ ]:
viz_opt_curves(BOUNDARIES_stats, BOUNDARIES_df_opt, method='insulation', chromnames=['chrX'], expected_mts=60000, stage='3-4h_repl_merged_5kb')

### Visualize boundaries:

In [ ]:
viz_tads(BOUNDARIES_df_opt, matrices, begin=300, end=500, ch='chrX', exp='3-4h_repl_merged_5kb', 
         resolution=5000, is_insulation=True, percentile=99.99)

### Compute insulation scores:

In [ ]:
stages_embryo = ['nuclear_cycle_12_repl_merged_5kb', 'nuclear_cycle_13_repl_merged_5kb', 'nuclear_cycle_14_repl_merged_5kb', '3-4h_repl_merged_5kb']

In [ ]:
BOUNDARIES_scores = insulation_scores(BOUNDARIES_df_opt, coolers, stages=stages_embryo, chromnames=['chrX'])

In [ ]:
BOUNDARIES_scores.head()

### Normalize insulation scores:

In [ ]:
BOUNDARIES_scores_norm = normalize(BOUNDARIES_scores, ['ins_score_{}'.format(x) for x in stages_embryo], type_norm='log-row')

In [ ]:
BOUNDARIES_scores_norm.head()

### Call clusters:

In [ ]:
BOUNDARIES_clustering = clusters(BOUNDARIES_scores_norm, ['norm_ins_score_{}'.format(x) for x in stages_embryo], 
                                 method='kmeans', n_clusters=3)

In [ ]:
BOUNDARIES_clustering.head()

### Evaluate clustering:

In [ ]:
silhouette(BOUNDARIES_clustering, ['norm_ins_score_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

### Visualize clusters:

In [ ]:
colors = clusters_dynamics(BOUNDARIES_clustering, ['norm_ins_score_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
_pca(BOUNDARIES_clustering, ['norm_ins_score_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
_tsne(BOUNDARIES_clustering, ['norm_ins_score_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
viz_tads(BOUNDARIES_clustering, matrices, begin=300, end=500, ch='chrX', exp='3-4h_repl_merged_5kb', 
         resolution=5000, method='kmeans', is_insulation=True, clusters=True, colors=colors, percentile=99.99)

# TADs

### Call TADs:

In [ ]:
TADs_stats, TADs_df, TADs_df_opt = domains(matrices, coolers, method='armatus', label='3-4h_repl_merged_5kb', 
                                           expected_tad_size=60000, grid=list(np.arange(0, 5, 0.01)), 
                                           chromnames=['chrX'], max_intertad=3, percentile=99.9, eps=1e-1)

# TADs_stats, TADs_df, TADs_df_opt = domains(matrices, coolers, method='modularity', label='3-4h_repl_merged_5kb', 
#                                            expected_tad_size=60000, grid=list(np.arange(0, 200, 0.1)), 
#                                            chromnames=['chrX'], max_intertad=2, percentile=99.9, eps=1e-1)

In [ ]:
TADs_df_opt.shape

### Visualize opt curves:

In [ ]:
viz_opt_curves(TADs_df, TADs_df_opt, method='modularity', chromnames=['chrX'], expected_mts=60000/5000, stage='3-4h_repl_merged_5kb')



### Visualize TADs:

In [ ]:
viz_tads(TADs_df_opt, matrices, begin=300, end=500, ch='chrX', exp='3-4h_repl_merged_5kb', 
         resolution=5000, is_insulation=False, percentile=99.99)

### Compute D-scores:

In [ ]:
stages_embryo = ['nuclear_cycle_12_repl_merged_5kb', 'nuclear_cycle_13_repl_merged_5kb', 'nuclear_cycle_14_repl_merged_5kb', '3-4h_repl_merged_5kb']

In [ ]:
TADs_scores = d_scores(TADs_df_opt, matrices, stages=stages_embryo)

In [ ]:
TADs_scores.head()

### Normalize D-scores:

In [ ]:
TADs_scores_norm = normalize(TADs_scores, ['D_{}'.format(x) for x in stages_embryo], type_norm='log-row')

In [ ]:
TADs_scores_norm.head()

### Call clusters:

In [ ]:
TADs_clustering = clusters(TADs_scores_norm, ['norm_D_{}'.format(x) for x in stages_embryo], 
                                 method='kmeans', n_clusters=3)

In [ ]:
TADs_clustering.head()

### Evaluate clustering:

In [ ]:
silhouette(TADs_clustering, ['norm_D_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

### Visualize clusters:

In [ ]:
colors = clusters_dynamics(TADs_clustering, ['norm_D_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
_pca(TADs_clustering, ['norm_D_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
_tsne(TADs_clustering, ['norm_D_{}'.format(x) for x in stages_embryo], 'cluster_kmeans')

In [ ]:
viz_tads(TADs_clustering, matrices, begin=300, end=500, ch='chrX', exp='3-4h_repl_merged_5kb', 
         resolution=5000, method='kmeans', is_insulation=False, clusters=True, colors=colors, percentile=99.99)